In [ ]:
This is the Jupyter Notebook Version of our python script at 
../offline-deployment/torchscript_exporter.py

In [ ]:
# 📓 TorchScript Exporter – Mini-Indig-LLM-Kit (Notebook Version)

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

In [ ]:
# 🔧 Config: Update these paths as needed
model_path = "../models/eastern_llm"
tokenizer_path = "../tokenizer/custom_tokenizer.json"  # or None to auto-load from model
output_path = "../offline-deployment/eastern_llm_torchscript.pt"
max_seq_len = 128

In [ ]:
# 🔄 Load Model
print(f"🔧 Loading model from {model_path}")
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    low_cpu_mem_usage=True,
)
model.eval()


In [ ]:
# 🧠 Tokenizer
if tokenizer_path and os.path.exists(tokenizer_path):
    print(f"📚 Loading tokenizer from {tokenizer_path}")
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, trust_remote_code=True)
else:
    print(f"📚 Loading tokenizer from model folder")
    tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
# 🤖 Build dummy input
example_input = tokenizer("Kenòn:we’s", return_tensors="pt", padding="max_length", max_length=max_seq_len)
input_ids = example_input["input_ids"]


In [ ]:
# 🎯 Trace and Save
print(f"📦 Saving TorchScript model to {output_path}")
traced = torch.jit.trace(model, (input_ids,))
torch.jit.save(traced, output_path)

print("✅ Done!")